In [1]:
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import sys
import seaborn as sns

In [2]:
df = pd.read_csv('data/data_by_month/jan_2023.csv')

In [4]:
# data cleaning
df.fillna(0, inplace=True)


,MONTH,OP_CARRIER_AIRLINE_ID,ORIGIN_AIRPORT_ID,ORIGIN_STATE_ABR,ORIGIN_STATE_NM,DEST_AIRPORT_ID,DEST_STATE_ABR,DEST_STATE_NM,DEP_DELAY,TAXI_OUT,ARR_DELAY,CANCELLED,DIVERTED,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
0,1,19393,10140,NM,New Mexico,10423,TX,Texas,0.0,0.0,0.0,1.0,0.0,0.0,619.0,0.0,0.0,0.0,0.0,0.0
1,1,19393,10140,NM,New Mexico,10423,TX,Texas,0.0,0.0,0.0,1.0,0.0,0.0,619.0,0.0,0.0,0.0,0.0,0.0
2,1,19393,10140,NM,New Mexico,10423,TX,Texas,-10.0,9.0,-14.0,0.0,0.0,77.0,619.0,0.0,0.0,0.0,0.0,0.0
3,1,19393,10140,NM,New Mexico,10423,TX,Texas,-9.0,11.0,-2.0,0.0,0.0,76.0,619.0,0.0,0.0,0.0,0.0,0.0
4,1,19393,10140,NM,New Mexico,10423,TX,Texas,-8.0,11.0,-20.0,0.0,0.0,80.0,619.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
573872,1,20500,15919,AR,Arkansas,13930,IL,Illinois,150.0,23.0,130.0,0.0,0.0,70.0,522.0,130.0,0.0,0.0,0.0,0.0
573873,1,20500,15919,AR,Arkansas,13930,IL,Illinois,214.0,49.0,221.0,0.0,0.0,68.0,522.0,0.0,0.0,7.0,0.0,214.0
573874,1,20500,15919,AR,Arkansas,13930,IL,Illinois,317.0,16.0,290.0,0.0,0.0,71.0,522.0,0.0,0.0,290.0,0.0,0.0
573875,1,20500,15919,AR,Arkansas,13930,IL,Illinois,377.0,48.0,389.0,0.0,0.0,79.0,522.0,0.0,4.0,12.0,0.0,373.0
